In [4]:
from ipynb.fs.full.LocationCleaningScript import clean_location
from ipynb.fs.full.cleanIndustries import cleanIndustries
# from ipynb.fs.full.CurrencyConverter import *
#from ipynb.fs.full.NormalizeWorkExperience import NormalizeExperience
import pandas as pd
import urllib.request
import json
from datascience import *
def RemoveSpacesFromTitles(df):
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df
def NormalizeExperience(filename):
    df = pd.read_csv(filename)
    df = df.replace(to_replace = ['5-7 years'], value = '5 - 7 years')
    df = df.replace(to_replace = ['1 year or less'], value = '0 - 1 years')
    return df
  

#x = x.to_df()


In [5]:
class currencyConverter:
    rates = {}
    def __init__(self, url):
        data = urllib.request.urlopen(url).read().decode('utf-8')
        f = json.loads(data)
        self.rates = f["rates"]
        
        
    def convert(self, amount, f, t):
        output = 0
        if f != 'USD':
            output = amount / self.rates[f] * self.rates['USD']
            return output
        return amount
        
def convertCurrency(df):
    for i in range(len(df['Currency'])):
        f = df['Currency'][i]
        salary = df['Annual_Salary'][i].replace(',', '')
        otherComp = df['Other_Compensation'][i]
        if not pd.isna(df['Other_Compensation'][i]):
            salary = float(salary) + float(otherComp)
        if f == 'Other':
            otherCur = df['Other'][i]
            if otherCur == 'Peso Argentino' or otherCur == 'Argentinian peso (ARS)' or otherCur == 'Argentine Peso':
                otherCur = 'ARS'
            if otherCur == 'Indian rupees' or otherCur == 'INR (Indian Rupee)':
                otherCur = 'INR'
            if otherCur == 'BRL (R$)' or otherCur == 'BR$':
                otherCur = 'BRL'
            if otherCur == 'Dkk' or otherCur == 'Danish Kroner' or otherCur == 'dkk':
                otherCur = 'DKK'
            if otherCur == 'Mexican pesos' or otherCur == 'Mexican Pesos':
                otherCur = 'MXN'
            if otherCur == 'Bdt':
                otherCur = 'BDT'
            if otherCur == 'American Dollars':
                otherCur = 'USD'
            if otherCur == 'Php' or otherCur == 'Philippine Peso' or otherCur == 'Philippine peso (PHP)' or otherCur == 'PhP (Philippine Peso)' or otherCur == 'Philippine Pesos':
                otherCur = 'PHP'
            if otherCur == 'PLN (Polish zloty)' or otherCur == 'Polish Złoty':
                otherCur = 'PLN'
            if otherCur == 'czech crowns' or otherCur == 'Czk':
                otherCur = 'CZK'
            if otherCur == 'Norwegian kroner (NOK)' or otherCur == 'Nok':
                otherCur = 'NOK'
            if otherCur == 'ILS/NIS' or otherCur == 'NIS (new Israeli shekel)' or otherCur == 'ILS (Shekel)' or otherCur == 'Israeli Shekels' or otherCur == 'Ils':
                otherCur = 'ILS'
            if otherCur == 'US Dollar':
                otherCur = 'USD'
            if otherCur == 'RMB (chinese yuan)' or otherCur == 'China RMB' or otherCur == 'RM':
                otherCur = 'CNY'
            if otherCur == 'Taiwanese dollars' or otherCur == 'NTD':
                otherCur = 'TWD'
            if otherCur == 'KRW (Korean Won)' or otherCur == 'Korean Won ':
                otherCur = 'KRW'
            if otherCur == 'Sgd':
                otherCur = 'SGD'
            if otherCur == 'AUD Australian' or otherCur == 'Australian Dollars ':
                otherCur = 'AUD'
            if otherCur == 'Euro':
                otherCur = 'EUR'
            if otherCur == 'THAI  BAHT':
                otherCur = 'THB'
            if otherCur == 'IDR ': # Whoever typed an extra space here is PURE EVIL
                otherCur = 'IDR'
            if otherCur in c.rates:
                df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), otherCur, 'USD'))
        else:
            if f == 'AUD/NZD':
                f = 'AUD'
            df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), f, 'USD'))
    return df

c = currencyConverter("http://data.fixer.io/api/latest?access_key=238f1925907e0f01a7fa1dc2b4042967")

In [7]:
table = NormalizeExperience('../2021_Results.csv')

table = RemoveSpacesFromTitles(table)

table = clean_location(table)
table = convertCurrency(table)
table = cleanIndustries(table)
#table
table.to_csv("CleanedData2021.csv")

<ipython-input-5-9176984daf03>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), f, 'USD'))
<ipython-input-5-9176984daf03>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), otherCur, 'USD'))
